# 01. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os

# 02. Import Data


In [2]:
path=r'/Users/nikolinastekovic/Library/CloudStorage/OneDrive-Personal/CF DATA ANALYICS/Immerse/4-Python/Instacart Basket Analysis'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_updated.pkl'))

In [6]:
ords_prods_merge.shape

(32404859, 19)

# 03. Task

## Step 2 - find the aggregated mean of the “order_number” column grouped by “department_id” for the entire dataframe

In [8]:
#calculate the mean of the “order_number” column grouped by the “department_id” column
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## Step 3 - How do the results for the entire dataframe differ from those of the subset? 

In [10]:
# Subset of first 1 million inputs
df_subset = ords_prods_merge[:1000000]

In [11]:
# the mean of the “order_number” column grouped by the “department_id” column for the subset
df_subset.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In the subset we can see first 1 milion inputs - this covers for only 8 departments. For these 8 departments the avarage is mostly higher then if we calculate the avarage on the whole dataframe (except the department_id: 17).

## Step 4 - Create a loyalty flag for existing customers

### transfrom()

In [12]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [13]:
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'Day_of_order',
       'order_hour_of_day', 'days_since_prior_order', 'new_customer',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', '_merge', 'price_range_loc',
       'busiest_day', 'busiest_days', 'busiest_period_of_day', 'max_order'],
      dtype='object')

### loc()

In [14]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [15]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [16]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [17]:
ords_prods_merge ['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

## Step 5 - Marketing Team wants to see the avarage of the price column grouped by loyalty category 

In [20]:
# Loyalty categoryies: Loyal Customer, Regular Customer, and New Customer

In [22]:
ords_prods_merge.groupby('loyalty_flag')['prices'].mean()

loyalty_flag
Loyal customer      10.386336
New customer        13.294670
Regular customer    12.495717
Name: prices, dtype: float64

## Step 6 - Create a spending flag for each user based on the average price across all their orders 

### transform()

Split user based on user_id, calculate mean product price for each user, create "price_mean" for aggregation results


In [23]:
ords_prods_merge['price_mean'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [24]:
ords_prods_merge.head(15)

,order_id,user_id,order_number,Day_of_order,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,price_mean
0,2539329,1,1,2,8,NaN,True,196,1,0,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797
1,2398795,1,2,3,7,15.0,False,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797
2,473747,1,3,3,12,21.0,False,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797
3,2254736,1,4,4,7,29.0,False,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797
4,431534,1,5,4,15,28.0,False,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797
5,3367565,1,6,2,7,19.0,False,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797
6,550135,1,7,1,9,20.0,False,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797
7,3108588,1,8,1,14,14.0,False,196,2,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797
8,2295261,1,9,1,16,0.0,False,196,4,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797
9,2550362,1,10,4,8,30.0,False,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797


### loc()

Create a flag that assigns a “spending” label to a user ID based on its corresponding mean_price

In [28]:
ords_prods_merge.loc[ords_prods_merge['price_mean'] < 10, 'spending_flag'] = 'Low spender'

In [29]:
ords_prods_merge.loc[ords_prods_merge['price_mean'] >= 10, 'spending_flag'] = 'High spender'

In [30]:
# Check frequencies of spending flags for accuracy
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

## Step 7 - Create an order frequency flag that marks the regularity of a user’s ordering behavior

### transform ()

Split user based on user_id, calculate “days_since_prior_order” median for each user, create "median_days'" for aggregation results


In [31]:
ords_prods_merge['median_days'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [36]:
ords_prods_merge.head()

,order_id,user_id,order_number,Day_of_order,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,price_mean,spending_flag,median_days
0,2539329,1,1,2,8,NaN,True,196,1,0,...,both,Mid-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,Low spender,20.5
1,2398795,1,2,3,7,15.0,False,196,1,1,...,both,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5
2,473747,1,3,3,12,21.0,False,196,1,1,...,both,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5
3,2254736,1,4,4,7,29.0,False,196,1,1,...,both,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5
4,431534,1,5,4,15,28.0,False,196,1,1,...,both,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5


### loc()

Create a flag that assigns a “order_frequency” label to a user ID based on its corresponding 'median_days'

In [38]:
ords_prods_merge.loc[ords_prods_merge['median_days'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [39]:
ords_prods_merge.loc[(ords_prods_merge['median_days'] > 10) & (ords_prods_merge['median_days'] <= 20), 'frequency_flag'] = 'Regular customer'

In [40]:
ords_prods_merge.loc[ords_prods_merge['median_days'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [41]:
# Check frequencies of ordering flags for accuracy
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

### Examining the null values

In [42]:
#create a subset of the dataframe containing only the values in question
df_ords_nan = ords_prods_merge[ords_prods_merge['frequency_flag'].isnull()==True]

In [43]:
df_ords_nan

,order_id,user_id,order_number,Day_of_order,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,price_mean,spending_flag,median_days,frequency_flag
13645692,895835,159838,1,0,17,NaN,True,10749,3,0,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17251990,895835,159838,1,0,17,NaN,True,33401,6,0,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
17622767,895835,159838,1,0,17,NaN,True,23695,2,0,...,Low-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
24138593,895835,159838,1,0,17,NaN,True,21334,5,0,...,Mid-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN
25880002,895835,159838,1,0,17,NaN,True,22198,1,0,...,Low-range product,Busiest day,Busiest days,Average orders,1,New customer,7.42,Low spender,NaN,NaN


All the Nan values are assigned to the same user-id in the same order. We can see max.order for this user is 1. Meaning, it is a new customer with only one user. Knowing this, we can replace the Nan value to Non-frequent customer

In [44]:
ords_prods_merge['frequency_flag'].fillna("Non-frequent customer", inplace = True)

In [45]:
ords_prods_merge['frequency_flag'].value_counts(dropna=False)

frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636442
Name: count, dtype: int64

## Step 9 - Export the dataframe

In [46]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_aggregated.pkl'))